# ***LLM***

In [3]:
import re

def filter_top_pattern(text: str) -> bool:
    """
    Filter text containing 'top' or 'Top' pattern using regex
    
    Args:
        text (str): Input text to check
        
    Returns:
        bool: True if pattern found, False otherwise
    """
    pattern = r'[tT]op'
    return bool(re.search(pattern, text))


In [4]:
import json

def load_report_config(file_path: str = "data/reports/alpha_report.json") -> dict:
    """
    Load report configuration from JSON file
    
    Args:
        file_path (str): Path to JSON config file
        
    Returns:
        dict: Loaded report configuration
    """
    with open(file_path, 'r') as f:
        return json.load(f)


report_config = load_report_config(r"D:\Desktop\arb-refactor\data\reports\alpha_report.json")


In [5]:
def get_abbreviation(report_config) -> str:
    
    abbreviated_functions = []

    for function_name, value in report_config.items():
        abbreviation = ', '.join(value['function']['abbreviation'])
        format_schema = f"- {function_name}: {abbreviation}"
        abbreviated_functions.append(format_schema)
    
    abbreviated_functions_to_string = "\n".join(abbreviated_functions)
    return abbreviated_functions_to_string

In [6]:
abbreviated_functions_to_string = get_abbreviation(report_config)

In [16]:
def get_function_description(report_config) -> str:
    function_descriptions = []
    for function_name, value in report_config.items():
        function_description = value['function']['description']
        format_schema = f"- {function_name}: {function_description}"
        function_descriptions.append(format_schema)
        
    function_descriptions_to_string = "\n".join(function_descriptions)
    return function_descriptions_to_string

In [17]:
function_descriptions_to_string = get_function_description(report_config)

In [21]:
import requests
import json

def get_user_prompt(query: str) -> str:
    prompt = """
        #📝Example requests and responses:
        
        Input: "I need to see the win/loss report from last week"
        Output: {{
            "function_called": "/winlost_detail"
        }}
        
        Input: "I want w/l report today"
        Output: {{
            "function_called": "/winlost_detail"
        }}

        Input: "Show me wl report"
        Output: {{
            "function_called": "/winlost_detail"
        }}

        Input: "Get WL please"
        Output: {{
            "function_called": "/winlost_detail"
        }}

        Input: "Show me the w-l details"
        Output: {{
            "function_called": "/winlost_detail"
        }}

        Input: "I need winlost stats"
        Output: {{
            "function_called": "/winlost_detail"
        }}

        Input: "I want to get the turnover report"
        Output: {{
            "function_called": "/turnover"
        }}

        Input: "I want to take turnover report for user 123"
        Output: {{
            "function_called": "/turnover"
        }}

        Input: "Get me the get winlost report for March transactions"
        Output: {{
            "function_called": "/winlost_detail"
        }}

        Input: "I want get performance of abc1 last week"
        Output: {{
            "function_called": "N/A"
        }}
        
        Input: "Hello how are you today?"
        Output: {{
            "function_called": "N/A"
        }}
        
        Input: "I want Sportsbook only"
        Output: {{
            "function_called": "N/A"
        }}
        
        Input: "I want change to a little bit, I want to get Product Virtual Sports and product detail Saba Basketball with user level Super Agent"
        Output: {{
            "function_called": "N/A"
        }}
        
        Input: "I want to get the top 20 outstanding"
        Output: {{
            "function_called": "/topoutstanding"
        }}
        
        Input: "The outstanding of Master1"
        Output: {{
            "function_called": "/outstanding"
        }}
        
        Input: "Top 40 Outstanding of Sportsbook"
        Output: {{
            "function_called": "/topoutstanding"
        }}
        
        Input: "My current outstanding"
        Output: {{
            "function_called": "/outstanding"
        }}
        
        Input: "Top 10 Outstanding of Sportsbook"
        Output: {{
            "function_called": "/topoutstanding"
        }}
        
        Input: "wl today please"
        Output: {{
            "function_called": "/winlost_detail"
        }}
        
        Based on this request, which function should be called? Return only the JSON response.
    """.format(query=query)
    
    return prompt


def functional_calling(query: str) -> str:
    # API endpoint
    url = "http://10.5.10.110:8090/api/chat"
    
    # Construct the system prompt using function metadata
    system_prompt = """
    You are an AI assistant that helps determine which function to call based on user input.
    Available functions:
        {function_description}
    - Try to recognize the function abbreviation from the user's request
    - Function Abbreviations:
        {abbreviation}
    Determine which function best matches the user's request and return it in JSON format like:
    {{
        "function_called": "/function_name"
    }}
    """.format(
        abbreviation=abbreviated_functions_to_string,
        function_description=function_descriptions_to_string
    )

    # Construct the prompt for function determination with examples
    user_prompt = get_user_prompt(query)

    
    format_schema =  {
        "type": "object",
        "properties": {
            "function_called": {
                "type": "string",
                "description": "The name of the function to call",
                "enum": [
                    '/winlost_detail',
                    '/turnover',
                    '/outstanding',
                    '/topoutstanding',
                    'N/A'
                ]
            }
        },
        "required": ["function_called"]
    }

    model = 'qwen2.5:14b'
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    headers = {"Content-Type": "application/json"}

    data = {
        "model": model,
        "messages": messages,
        "format": format_schema,
        "stream": False
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    return json.loads(response.json()['message']['content'])

In [ ]:

# Example usage
example_inputs = queries = [
    "Get me a Win Loss Detail Report yesterday",
    "Get me a Turnover Detail Report on day 10",
    "Net Turnover Detail Report from 1/1 to 31/1",
    "Get me a Win Loss Detail Report for Direct Member who played Product Detail Sportsbook in Sportsbook Product from 01/02/2024 to 15/02/2024",
    "Give me my Win Loss Detail report  last week",
    "Give me a Gross Commission Report Detail report for Sportsbook from last week.",
    "Give me my Win Loss Detail report  last week",
    "Get me a Company Report of master1",
    "Win/Loss details for Product Sportsbook",
    "Get me a Win Loss on day 10",
    "I want Win Loss report of Sportsbook this week",
    "I want get performance of abc1 last week",
    "Live Casino Master report of Master master1 yesterday",
    "I need Member report of Agent agent01 last week",
    "Show me today's summary",
    "How did we perform in live casino last week?",
    "The amount I need to pay for master abc1 last week",
    "Get report of master1",
    "Win/Loss details for Product Sportsbook",
    "Get me a Super Report of master1"
]

for input_text in example_inputs:
    called_function = functional_calling(input_text)
    print(f"Input: {input_text}")
    print(f"Called function: {called_function['function_called']}")
    print("\n")


In [28]:
functional_calling("turnover today please")

{'function_called': '/winlost_detail'}

In [2]:
functional_calling("Hello")

{'function_called': 'N/A'}

In [3]:
functional_calling("I want Sportsbook only")

{'function_called': 'N/A'}

In [3]:
functional_calling("give me top 20 outstanding")

{'function_called': '/get_topoutstanding_report'}

In [4]:
functional_calling("give me outstanding")

{'function_called': '/get_outstanding_report'}

In [5]:
functional_calling("I want to get the list of leading 20 outstanding")

{'function_called': '/get_topoutstanding_report'}

In [6]:
functional_calling("give me the first 20 outstanding sorting from highest to lowest")

{'function_called': '/get_topoutstanding_report'}

In [7]:
functional_calling("give me the first 20 outstanding decreasing")

{'function_called': '/get_topoutstanding_report'}

In [10]:
functional_calling("I want to get turnover report")

{'function_called': '/get_turnover_report'}

In [11]:
functional_calling("Do you know donald trump?")

{'function_called': 'N/A'}

In [12]:
functional_calling("How many outstanding do you have?")

{'function_called': '/get_outstanding_report'}

# ***Embedding data***

In [29]:
from sentence_transformers import SentenceTransformer
import torch

# Download from the 🤗 Hub
model_embedding = SentenceTransformer("hiieu/halong_embedding")

In [30]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model_embedding.to(device)

In [52]:
import faiss
import tqdm
import numpy as np

def faiss_indexing(data_embeddings, path_save):
    # indexing
    cpu_index = faiss.IndexFlatIP(768)
    for embedding in tqdm.tqdm(data_embeddings, colour='green', desc='Indexing'):
        embedding = embedding.astype(np.float32).reshape(1, -1)
        cpu_index.add(embedding)

    # Save vector database
    faiss.write_index(cpu_index, path_save)

In [66]:
import pickle

def bm25_indexing(contents, path_save):

  tokenized_corpus = []
  for doc in tqdm.tqdm(contents, colour='green', desc='Indexing'):
    tokenized_corpus.append(doc)

  with open(path_save, 'wb') as f:
    pickle.dump(tokenized_corpus, f)


In [86]:
abbreviated_functions_to_dict = {}
abbreviated_functions_index = {}
contents = []
functions = []
for function_name, value in report_config.items():
    abbreviation = ', '.join(value['function']['abbreviation'])
    abbreviated_functions_to_dict[function_name] = value['function']['abbreviation']
    contents.extend(value['function']['abbreviation'])
    functions.extend([function_name] * len(value['function']['abbreviation']))

In [55]:
embeddings = model.encode(contents)
faiss_indexing(embeddings, 'abbreviation_faiss.index')

Indexing: 100%|██████████| 36/36 [00:00<00:00, 36227.19it/s]


In [68]:
bm25_indexing(contents, 'abbreviation_bm25.pkl')

Indexing: 100%|██████████| 36/36 [00:00<?, ?it/s]


In [69]:
def load_bin(path):
    cpu_index = faiss.read_index(path)
    return cpu_index

def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [70]:
faiss_index = load_bin('./abbreviation_faiss.index')
bm25_index = load_pickle('./abbreviation_bm25.pkl')

In [98]:
from rank_bm25 import BM25Okapi

def bm25_keyword_search(query, bm25_index, topk=1):

  bm25 = BM25Okapi(bm25_index)
  docs = bm25.get_top_n(query, bm25_index, n=topk)
  index = np.where(np.isin(contents, docs))[0].tolist()
  return np.array(functions)[index].tolist()

In [103]:
bm25_keyword_search(
    query="wl please",
    bm25_index=bm25_index,
    topk=4
)

['/outstanding', '/outstanding', '/topoutstanding', '/topoutstanding']

In [119]:
def semantic_search(query, faiss_index, topk):
  prompt_embedding = model_embedding.encode([query])
  prompt_embedding = np.array(prompt_embedding)
  scores, indices = faiss_index.search(prompt_embedding, topk)
  print(scores)
  return np.array(functions)[indices].tolist()[0]

In [120]:
semantic_search(
    query="wl please",
    faiss_index=faiss_index,
    topk=4
)

[[0.4937626  0.45274523 0.4524747  0.41575557]]


['/winlost_detail', '/winlost_detail', '/winlost_detail', '/winlost_detail']

In [107]:
def hybrid_search(query, topk):
  contexts = semantic_search(query, faiss_index, topk)
  contexts.extend(bm25_keyword_search(query, bm25_index, topk))
  return contexts

In [108]:
from sentence_transformers import CrossEncoder
import torch.nn as nn

model_reranking = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512)

sigmoid = nn.Sigmoid()

def hybrid_search_with_reranking(query, topk):
  contexts = hybrid_search(query, topk)
  formated_contexts = [[query, context] for context in contexts]

  scores = model_reranking.predict(formated_contexts, activation_fct=sigmoid)
  top_k_values = np.sort(scores)[-topk:][::-1]

  top_k_indices = np.argsort(scores)[-topk:][::-1]
  print(top_k_values)

  contexts = np.array(contexts)
  best_contexts = contexts[top_k_indices].tolist()

  return best_contexts

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

c:\Users\leon.pham\AppData\Local\anaconda3\envs\langchain\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\leon.pham\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [121]:
hybrid_search_with_reranking(
    query="wl",
    topk=4
)

[[0.6948472  0.59056854 0.5153476  0.50300705]]
[3.5108358e-05 2.7640401e-05 2.7640401e-05 2.7640401e-05]


['/turnover', '/winlost_detail', '/winlost_detail', '/winlost_detail']

In [ ]:
keyword + faiss -> OKE